# Imports

In [ ]:
import numpy as np
import pandas as pd

from keras.layers import Input,Dense,Flatten,Reshape
from keras.models import Sequential,Model
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Data

In [ ]:
filename = "330_Days_Data.csv"
dataset = pd.read_csv(filename,sep=",")
data = np.array(dataset)
data = data[:,1:]

n_features = np.shape(data)[1]
n_days = int(filename.split("_")[0])
n_days_used = 30
n_samples = int(len(data)/n_days)
percentage = 0.7
train_data = data[:int(n_samples*percentage)*n_days]
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(train_data)

train_data = scaler.transform(train_data)
data_dim = [n_days_used,n_features]
train_data = np.reshape(train_data,(-1,data_dim[0]*(data_dim[1])))
minibatch = np.zeros((11,7,205,270))
for sample in range(len(train_data)):
    minibatch[sample%11][int(sample/(11*205))][int((sample/11)%205)] = train_data[sample]

# GAN

In [ ]:
class GAN():
    def __init__(self):
        # Data and Noise dimensions
        self.data_rows = 1
        self.data_coll = 9
        self.data_shape = (self.data_rows,self.data_coll)
        self.latent_dim = 20
        
        # Using Adam Optimizer with lr = 0.0005
        optimizer = Adam(0.0005)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss ='binary_crossentropy',optimizer=optimizer,metrics=['accuracy']) 
        self.generator = self.build_generator()
        
        # The generator transforms noise in data 
        z = Input(shape=(self.latent_dim,))
        data = self.generator(z)
        
        # Train only the generator on the combined model
        self.discriminator.trainable = False
        
        valid_or_not = self.discriminator(data)
        
        self.comb = Model(z, valid_or_not)
        self.comb.compile(loss='binary_crossentropy',optimizer=optimizer)
    
    def build_generator(self):
        
        model = Sequential()
        model.add(Dense(256, activation = "relu", input_dim = self.latent_dim))
        model.add(Dense(512, activation = "relu"))
        model.add(Dense(9, activation = "sigmoid"))
        model.add(Reshape(self.data_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        data = model(noise)
        
        return Model(noise, data)
    
    def build_discriminator(self):
        
        model = Sequential()
        model.add(Flatten(input_shape = self.data_shape))
        model.add(Dense(512, activation = "relu"))
        model.add(Dense(256, activation = "relu"))
        model.add(Dense(1, activation = "sigmoid"))
        model.summary()
        
        data = Input(shape = self.data_shape)
        result = model(data)
        
        return Model(data, result)
    
    def save_data(self, global_step, file_name):
        gen_noise = np.random.normal(0,1,(387450, self.latent_dim))
        gen_data = self.generator.predict(gen_noise)
        generated_sample = np.reshape(gen_data,(387450,9))
        np.savetxt("generated_samples_"+file_name+"_"+str(global_step)+".csv",generated_sample,delimiter = ",")
    
    def train(self, global_steps, file_name, batch_size = 205, checkpoint=1000):
            # The labels for original data are 1 and the labels for fake data are 0
            original = np.ones((batch_size, 1))
            fake = np.zeros((batch_size, 1))
            
            for global_step in range(global_steps):
                
                gen_noise = np.random.normal(0, 1, (205*30,self.latent_dim))
                idx = np.random.randint(0,7)
                # Use only data from the last month class
                o_data = minibatch[10][idx]
                noise = gen_noise
                o_data = np.reshape(o_data, newshape = [-1,1,9])
                
                gen_data = self.generator.predict(noise)
                
                # Train the discriminator network
                d_loss_real = self.discriminator.train_on_batch(o_data, original)
                d_loss_fake = self.discriminator.train_on_batch(gen_data, fake)
                d_total_loss = np.add(d_loss_real, d_loss_fake)/2
                
                # Training the Generator
                # The generator wants it's labels to be 1 so it can fool the discriminator
                g_loss = self.comb.train_on_batch(noise, original) 
                
                print("Global Step :%d | Discriminator loss : %f | Generator loss : %f | Discriminator Acc : %f"%(global_step, d_total_loss[0], g_loss, 100*d_total_loss[1]))
                
                # Save sintetic samples
                if (global_step % checkpoint == 0):
                    self.save_data(global_step,file_name)
                
                
                

# Run 

In [ ]:
for i in range(5):
    gan = GAN()
    gan.train(global_steps = 50001, file_name = str(i), batch_size = 205*30, checkpoint = 10000)